In [1]:
disease_arr=['Acne and Rosacea Photos','Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions','Atopic Dermatitis Photos','Bullous Disease Photos']

In [2]:
import os
import cv2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D,MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing import image
import pandas as pd
import tensorflow as tf
import keras
import pickle

In [6]:
def data_dictionary():
    path_train="train_dataset"
    path_test="test_dataset"
    train_dictionary={"image_path":[],"target":[]}
    test_dictionary={"image_path":[],"target":[]}
    k=0
    for i in disease_arr:
        path_disease_train=path_train+"/"+i
        path_disease_test=path_test+"/"+i
        image_list_train=os.listdir(path_disease_train)
        image_list_test=os.listdir(path_disease_test)
        for j in image_list_train:
            img_path_train=path_disease_train+"/"+j
            train_dictionary["image_path"].append(img_path_train)
            train_dictionary['target'].append(k) 
        for m in image_list_test :
            img_path_test=path_disease_test+"/"+m
            test_dictionary["image_path"].append(img_path_test)
            test_dictionary['target'].append(k)
        k+=1 
    test_df=pd.DataFrame(test_dictionary)
    train_df=pd.DataFrame(train_dictionary)
        
    return  train_df,test_df            

In [7]:
def load_data(input_size=(100,100)):
    images=[]
    images2=[]
    train_df,test_df=data_dictionary()
    for i in train_df['image_path']:
        img=cv2.imread(i)
        img=cv2.resize(img,input_size)
        images.append(img)
    y_train=np.asarray(train_df['target'])
    x_train=np.asarray(images)
    for i in test_df['image_path']:
        img=cv2.imread(i)
        img=cv2.resize(img,input_size)
        images2.append(img)
    y_test=np.asarray(test_df['target'])
    x_test=np.asarray(images2)
    return x_train,x_test,y_train,y_test

In [8]:
x_train,x_test,y_train,y_test=load_data(input_size=(100,100))

In [11]:
from tensorflow.keras.applications import  VGG19
from tensorflow.keras.models import Model
from keras.applications.vgg19 import preprocess_input

In [12]:
vgg16 = VGG19(include_top=False,weights='imagenet')

80134624/80134624 [==============================] - 21s 0us/step


In [13]:
import joblib

In [14]:
filename="vgg16.sav"
joblib.dump(vgg16,open(filename,'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\conv2d_10
......vars
.........0
.........1
...layers\conv2d_11
......vars
.........0
.........1
...layers\conv2d_12
......vars
.........0
.........1
...layers\conv2d_13
......vars
.........0
.........1
...layers\conv2d_14
......vars
.........0
.........1
...layers\conv2d_15
......vars
.........0
.........1
...layers\conv2d_2
......vars
.........0
.........1
...layers\conv2d_3
......vars
.........0
.........1
...layers\conv2d_4
......vars
.........0
.........1
...layers\conv2d_5
......vars
.........0
.........1
...layers\conv2d_6
......vars
.........0
.........1
...layers\conv2d_7
......vars
.........0
.........1
...layers\conv2d_8
......vars
.........0
.........1
...layers\conv2d_9
......vars
.........0
.........1
...layers\input_layer
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
....

In [15]:
vgg19=joblib.load("vgg16.sav")

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-01-11 21:31:22        11992
metadata.json                                  2023-01-11 21:31:22           64
variables.h5                                   2023-01-11 21:31:24     80153976
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\conv2d_10
......vars
.........0
.........1
...layers\conv2d_11
......vars
.........0
.........1
...layers\conv2d_12
......vars
.........0
.........1
...layers\conv2d_13
......vars
.........0
.........1
...layers\conv2d_14
......vars
.........0
.........1
...layers\conv2d_15
......vars
.........0
.........1
...layers\conv2d_2
......vars
.........0
.........1
...layers\conv2d_3
......vars
.........0
.........1
...layers\conv2d_4
......vars
.........0
.........1
...layers

In [16]:
train_img=preprocess_input(x_train)
test_img=preprocess_input(x_test)

In [17]:
features_train=vgg16.predict(train_img)

92/92 [==============================] - 278s 3s/step


In [18]:
features_test=vgg16.predict(test_img)

27/27 [==============================] - 56s 2s/step


In [19]:
num_train=x_train.shape[0]
num_test=x_test.shape[0]

In [20]:
x_train=features_train.reshape(num_train,4608)
x_test=features_test.reshape(num_test,4608)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten, Dense

In [27]:
model=Sequential([ Dense(1024,activation='relu'),
                   Dense(512,activation='relu'),
                  Dense(256,activation='relu'),
                  Dense(128,activation='relu'),
                  Dense(6,activation='sigmoid'),

])

In [28]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [29]:
model.fit(x_train,y_train,batch_size=32,epochs=14,validation_data=(x_test,y_test))

Epoch 1/14
92/92 [==============================] - 12s 117ms/step - loss: 1.8884 - accuracy: 0.5612 - val_loss: 1.0210 - val_accuracy: 0.6184
Epoch 2/14
92/92 [==============================] - 11s 115ms/step - loss: 0.5563 - accuracy: 0.7939 - val_loss: 0.6641 - val_accuracy: 0.7428
Epoch 3/14
92/92 [==============================] - 10s 114ms/step - loss: 0.2870 - accuracy: 0.8978 - val_loss: 0.6556 - val_accuracy: 0.7895
Epoch 4/14
92/92 [==============================] - 10s 114ms/step - loss: 0.1679 - accuracy: 0.9460 - val_loss: 0.7932 - val_accuracy: 0.7919
Epoch 5/14
92/92 [==============================] - 11s 117ms/step - loss: 0.0790 - accuracy: 0.9754 - val_loss: 0.8352 - val_accuracy: 0.8170
Epoch 6/14
92/92 [==============================] - 11s 114ms/step - loss: 0.0732 - accuracy: 0.9788 - val_loss: 1.0462 - val_accuracy: 0.8074
Epoch 7/14
92/92 [==============================] - 11s 115ms/step - loss: 0.1412 - accuracy: 0.9607 - val_loss: 0.8771 - val_accuracy: 0.7763

In [31]:
model.save("skin_disease_predictor.h5")